# Data pre-processing

In [ ]:
import wfdb
import numpy as np
import os
import matplotlib.pyplot as plt
from IPython.display import display

## Segmenting the data from ECG DB into 3 min segments all listed as arrays

### Read all the ECG signal files being used (a01-a05 and c01-c05) into one list

In [ ]:
ECGFiles = [] # An List with all the ECG Signal filenames
for root, dirs, files in os.walk(r'C:\Python27\Notebooks\wfdb\apneaecg\Labelled1'):
    for file in files:
        if file.endswith('.dat'):
            parts = file.split('.')  #split the file using . as separator
            ECGFiles.append(parts[0])  # append only the first part of the file without the extension

In [ ]:
len(ECGFiles), ECGFiles

### Take each ECG file and convert it into a list of arrays of 3 minute segments
(Note each segment overlaps with the next 2 segments)

In [ ]:
 cd "C:/Python27/Notebooks/wfdb/apneaecg/Labelled1/"

In [ ]:
def ECG_Segment(ECG_Signal_file):
    record = wfdb.rdsamp(ECG_Signal_file)
    bfunc = np.asarray(record.p_signals)
    rawECGFunc = []
    for i in range((bfunc.size / 6000)-2):
        rawECGFunc.append(bfunc[i*6000:((i*6000)+18000)])
    return rawECGFunc



In [ ]:
# Create list containing the data
AllDataSegmented = []
for i in ECGFiles:
    Seg = ECG_Segment(i)
    AllDataSegmented.append(Seg)


In [ ]:
# We should have 35 arrays of segmented data
len(AllDataSegmented)

In [ ]:
L = AllDataSegmented[0][5][16000:17000,0]
plt.plot(L)
plt.show()

### Now we have all our segments below details how to use them

In [ ]:
# As seen above we have 35 elements in the list AllDataSegmented
# To extract all the segments for one ECG file use AllDataSegmented[X] eg
b = AllDataSegmented[4]

# To extract one of the 3 min segments from this ECG signal use b = AllDataSegmented[X][X]
d = AllDataSegmented[4][3][12000:18000]

# to extract one section eg 60seconds of one of the 3 min segments of 1 of the signals use b = AllDataSegmented[X][X][0:6000]
e = AllDataSegmented[1][3][0:18000]

# To extract 1 3 min segment as a flat array rather than an array of lists containing 1 value
L = AllDataSegmented[1][3][:18000,0]

In [ ]:
d.shape

In [ ]:
plt.plot(d)
plt.show()

# Calc RRIntervals for the segments

## Try putting High pass filter in front of the RR Peak detection

In [ ]:
cd "C:\Python27\Notebooks\wfdb"

In [ ]:
L = AllDataSegmented[4][3][:18000,0]    

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
import matplotlib.pyplot as plt
def sine_generator(fs, sinefreq, duration):
    T = duration
    nsamples = fs * T
    w = 2. * np.pi * sinefreq
    t_sine = np.linspace(0, T, nsamples, endpoint=False)
    y_sine = np.sin(w * t_sine)
    result = pd.DataFrame({ 
        'data' : y_sine} ,index=t_sine)
    return result

#High Pass filter

def butter_highpass(cutoff, fs, order=15):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=1):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y


### Demonstrating the high pass filter on a signal with 5Hz and 1Hz elements

In [ ]:

fps = 100
sine_fq = 5 #Hz
duration = 5 #seconds
sine_5Hz = sine_generator(fps,sine_fq,duration)
sine_fq = 1 #Hz
duration = 5 #seconds
sine_1Hz = sine_generator(fps,sine_fq,duration)

sine_fq = 40 #Hz
duration = 5 #seconds
sine_40Hz = sine_generator(fps,sine_fq,duration)

sine = sine_5Hz + sine_1Hz + sine_40Hz

filtered_sine = butter_highpass_filter(sine.data,20,fps)

plt.figure(figsize=(20,10))
plt.subplot(211)
plt.plot(range(len(sine)),sine)
plt.title('generated signal')
plt.subplot(212)
plt.plot(range(len(filtered_sine)),filtered_sine)
plt.title('filtered signal')
plt.show()

In [ ]:
# Filtering using high pass filter to allow any frequency above 0.5 
# should remove respiratory artifacts (0.12Hz to 0.50HZ) ie 8-30 breaths per minute
#### BUT found that 20Hz cut off gave better suppression of non-R peak aspects of the ECG

# Trying data filtering
t_ECG = np.linspace(0,10, 1000, endpoint=False)
ECG = pd.DataFrame({'data' : L}, index=t_ECG)

filtered_ecg = butter_highpass_filter(ECG.data,20,fps)   # 20 here is the 20Hz cutoff

plt.figure(figsize=(20,10))
plt.subplot(211)
plt.plot(range(len(ECG)),ECG)
plt.title('generated signal')
plt.subplot(212)
plt.plot(range(len(filtered_ecg)),filtered_ecg)
plt.title('filtered signal')
plt.show()

In [ ]:
# Plot an example of one of the data segments and its filtered version

plt.figure(figsize=(20,10))
plt.subplot(211)
plt.plot(range(len(AllDataSegmented[16][3])),AllDataSegmented[16][3])
plt.title('generated signal')
plt.subplot(212)
plt.plot(range(len(AllDataFiltered[16][3])),AllDataFiltered[16][3])
plt.title('filtered signal')
plt.show()

## Lets only take the mid sections of the data - cleaner data

In [ ]:
MidDataSegments = []
for i in AllDataSegmented:
    MidSeg = i[75:(len(i))-75]
    MidDataSegments.append(MidSeg)

In [ ]:
#L = MidDataSegments[0][1][10000:12000,0]
L = MidDataSegments[0][5]
plt.plot(L)
plt.show()

## Lets run the filter against all our data segments
(Remember that each ECG signal is for a different total time length so each MidDataSegmented will have a different length len)

In [ ]:
AllDataFiltered = []
t_ECG = np.linspace(0,180, 18000, endpoint=False)    # Setup the index for the dataframe used
for i in MidDataSegments:
    SegmentsFiltered = []
    for j in i:
        TempSig = j[:18000,0]
        ECG = pd.DataFrame({'data' : TempSig}, index=t_ECG)
        filtered_ecg = butter_highpass_filter(ECG.data,20,fps)
        SegmentsFiltered.append(filtered_ecg)
    AllDataFiltered.append(SegmentsFiltered)

### Might be a good time to dump the data into a dataframe and then into a pickle file which can be opened from this stage then without rerunning all above

In [ ]:
#Create pkl file represenation of filtered data (Note it's 2.2GB in size)

import joblib
iterate =0
for i in AllDataFiltered:
    filename = 'AllDataFiltered'
    filenumber = str(iterate)
    fileextension = '.pkl'
    name = filename+filenumber+fileextension
    joblib.dump(AllDataFiltered[iterate], name, compress=3)  
    #print(name)
    iterate = iterate + 1


In [ ]:
AllDataFiltered = joblib.load('AllDataFiltered.pkl')

In [ ]:
# Load pickle file fitlered data

import joblib
iterate = 0
AllDataFilteredTest = []
while iterate < 35:
    filename = 'AllDataFiltered'
    filenumber = str(iterate)
    fileextension = '.pkl'
    name = filename+filenumber+fileextension
    AllDataFilteredTest.append(joblib.load(name))
    iterate = iterate + 1




### Heart rate calculator using threshold method


In [ ]:
def RRInterval(SegmentIn):
    overThres = [] 
    mean = SegmentIn.mean()
    std = SegmentIn.std()
    Thres = mean + (2 * std) # use criteria that peaks are at least 2 X std deviation above mean
    print Thres
    for i in SegmentIn:
        overThres.append(1*(i>Thres))    # Find those over the threshold 
    overThresnp = np.array(overThres)
    changes = overThresnp[2:18000]-overThresnp[1:17999]
    ndx = []
    for i in range(17998):
        if changes[i] > 0:
            ndx.append(i+2) 
    ndxlen=len(ndx)
    npndx =np.array(ndx)
    RRIntervals = npndx[2:ndxlen]-npndx[1:ndxlen-1]
    lenRR = len(RRIntervals)
    HRs = []
    for i in range(lenRR):
        HRs.append (6000/RRIntervals[i])
    return RRIntervals, HRs

### Correction function

In [ ]:
def RRCorrection (RRIntervalValues):
    Anp = np.array(RRIntervalValues)
    Amean = Anp.mean()
    A = RRIntervalValues
    for c in range(len(A)):
       if A[c] > Amean+(Amean/3):
        print A[c]
        A[c] = Amean
        print A[c]
       if A[c] < Amean-(Amean/3):
        print A[c]
        A[c] = Amean
        print A[c]
    return A

### Use this function to run the RRInterval and Heart Rate calulator and the correction functions on the data

In [ ]:
def GetHeartRates (FilteredSig):
    AllHeartRates = []
    AllRRs = []
    SegmentRRs = []
    SegmentHRs = []
    iterate2 = 0
    for j in FilteredSig:
        print("...")
        print(iterate2)
        iterate2 = iterate2 + 1
        TempSig = j[:18000]
        TempRR, TempHR = RRInterval(TempSig)
        TempCorrectedHR = RRCorrection(TempHR)
        TempCorrectedRRInt = RRCorrection(TempRR)
        SegmentHRs.append(TempCorrectedHR)
        SegmentRRs.append(TempCorrectedRRInt)
    AllHeartRates.append(SegmentHRs)
    AllRRs.append(SegmentRRs)
    return AllRRs, AllHeartRates

### Use above funtion to get HeartRates for signals to be used for trainign the data 5 signals from Apnea patients a01-a05 and 5 from control set c01-05

In [ ]:
# These take some time to run
RRa01, HRa01 = GetHeartRates (AllDataFiltered[0])

In [ ]:
RRa02, HRa02 = GetHeartRates (AllDataFiltered[1])

In [ ]:
RRa03, HRa03 = GetHeartRates (AllDataFiltered[2])

In [ ]:
RRa04, HRa04 = GetHeartRates (AllDataFiltered[3])

In [ ]:
RRa05, HRa05 = GetHeartRates (AllDataFiltered[4])

In [ ]:
RRc01, HRc01 = GetHeartRates (AllDataFiltered[5])

In [ ]:
RRc02, HRc02 = GetHeartRates (AllDataFiltered[6])

In [ ]:
RRc03, HRc03 = GetHeartRates (AllDataFiltered[7])

In [ ]:
RRc04, HRc04 = GetHeartRates (AllDataFiltered[8])

In [ ]:
RRc05, HRc05 = GetHeartRates (AllDataFiltered[9])

In [ ]:
# Lets dump this stage of data - do for each set of HR data
import joblib
joblib.dump(HRc01[0], 'HRc01.pkl', compress=3)  

### Statistical calculations RRIntervals NN50 values

In [ ]:
'''Calc NN50 measure (variant 1), defined as the number of pairs of adjacent RR- intervals 
where the first RR- interval exceeds the second RR- interval by more than 50 ms.
Note my calc is not strictly NN50 because the resolution doesn't show if it's 51ms only if it's 60ms greater
'''

def NN50 (RRintervals):
    NN50_count = 0
    temp = 999
    for i in RRintervals:
        if (temp - i) > 5:
            NN50_count = NN50_count + 1
        temp = i
    return NN50_count


In [ ]:
'''The NN50 measure (variant 2), defined as the number of pairs of adjacent RR-intervals where the 
second RR- interval exceeds the first RR interval by more than 50 ms
'''
def NN50_count2 (RRint):
    NN50_count2 = 0
    temp = 999
    for i in RRint:
        if (i - temp) > 5:
            NN50_count2 = NN50_count2 + 1
        temp = i
    return NN50_count2


#### We want to add stats to each stats array by giving this function our arrays of RR-Intervals and HRs and the arrays for the stats

In [ ]:
def HRstats_new(RRArray, HRArray,HRm,HRs,NN501,NN502):
    for i in HRArray:
        #print (i)
        HRm.append(np.mean(i))
        HRs.append(np.std(i))
    for i in RRArray:
        NN501.append(NN50(i))
        NN502.append(NN50_count2(i))
    return HRm,HRs,NN501,NN502

# Create datafeatures and classifys buckets for training and validating the classifier then add the HR stats and the annotations

In [ ]:
datafeatures = []        #Hashing this out to allow appending the next lot of data
classifys = []           #Hashing this out to allow appending the next lot of data

In [ ]:
def prepclassifiercontainers(datafeatures, classifys):
    iterate = 0
    if len(MyAnn) < len(HRmean):
        for i in MyAnn:
            data_features = []
            data_features.append(HRmean[iterate])
            data_features.append(HRstd[iterate])
            data_features.append(NN50Result[iterate])
            data_features.append(NN502Result[iterate])
            classifys.append(MyAnn[iterate])
            iterate=iterate+1
            datafeatures.append(data_features)
    else:
        for i in HRmean:                 
            data_features = []
            data_features.append(HRmean[iterate])
            data_features.append(HRstd[iterate])
            data_features.append(NN50Result[iterate])
            data_features.append(NN502Result[iterate])
            classifys.append(MyAnn[iterate])
            iterate=iterate+1
            datafeatures.append(data_features)
    return datafeatures, classifys

## Function below to pull in the annotations for the 

In [ ]:
def Annot(sigfile):
    annotation = wfdb.rdann(sigfile, 'apn')
    AnnSym = annotation.symbol
    MyAnn = [] # initialise the container
    for i in range (len(AnnSym)-149):             # Using only the mid section of the signal so taking 75 minutes off each end 
        if AnnSym[i+74] == 'A' and AnnSym[i+75] == 'A' and AnnSym[i+76] == 'A': 
            Ann = 'A'
        else :
            Ann = 'N'
        MyAnn.append(Ann)
    return MyAnn

### Run the step below agaisnt each signal file to populate the datafeatures and classifys (labels) arrays

In [ ]:
HRmean = []
HRstd = []
NN50Result = []
NN502Result = []
HRstats_new(RRc05[0], HRc05[0], HRmean, HRstd, NN50Result, NN502Result) 
        ## Change this  to the signal file you're populating featuress from
MyAnn = Annot('c05')   ## Change this each time to the signal file you're populating labels from
datafeatures, classifys = prepclassifiercontainers(datafeatures, classifys)


In [ ]:
 len(datafeatures), len(classifys), datafeatures,  classifys

## Shuffle the data before using it for training

In [ ]:
iterate = 0
for i in datafeatures:
    i.append(classifys[iterate])
    iterate = iterate + 1

In [ ]:
import random
random.shuffle(datafeatures)

In [ ]:
classifys = []
iterate = 0
for i in datafeatures:
    classifys.append(i[4])
    i.pop(4)

In [ ]:
import pickle
filename = 'classifys_shuffled.pkl'
pickle.dump(classifys, open(filename, 'wb'))
filename = 'datafeatures_shuffled.pkl'
pickle.dump(datafeatures, open(filename, 'wb'))

In [ ]:
import pickle
filename = 'classifys_shuffled.pkl'
classifys = pickle.load(open(filename, 'rb'))
filename = 'datafeatures_shuffled.pkl'
datafeatures = pickle.load(open(filename, 'rb'))

# Lets try classification

## Do the initial classifer training and testing

#### Split the data into a training and test set 90:10

In [ ]:
dflen = len(datafeatures)
split = (9*dflen/10)            # Split is 9/10 of the total dataset

###Running the SVM (support vector model)

In [ ]:
#Running the SVM (support vector model)
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

svm = SVC(C=1.0, gamma='auto', kernel='rbf')
svm.fit(datafeatures[:split], classifys[:split])        # We're training on just the data features up to the split

In [ ]:
# Make an array of predictions on the validation set
predictions = svm.predict(datafeatures[split+1:dflen-1])        # getting predictions based on the held back validation

# Output the percentage accuracy, total predicted correctly and the confusion matrix for each model
print(svm.score(datafeatures[split+1:dflen-1], classifys[split+1:dflen-1]))
print(confusion_matrix(predictions, classifys[split+1:dflen-1]))

## Save the svm prediction engine to file

In [ ]:
cd "C:\Python27\Notebooks\wfdb\apneaecg\Labelled"

In [ ]:
import pickle
filename = 'finalsvm.pkl'
pickle.dump(svm, open(filename, 'wb'))

In [ ]:
import pickle
filename = 'datafeatures.pkl'
pickle.dump(datafeatures, open(filename, 'wb'))

In [ ]:
import pickle
filename = 'classifys.pkl'
pickle.dump(classifys, open(filename, 'wb'))

In [ ]:
import pickle
filename = 'finalsvm.pkl'
svm2 = pickle.load(open(filename, 'rb'))
#print(svm2.score(validation_data_features, validation_classifys))

# Try different classifiers

In [ ]:
import pickle
filename = 'classifys.pkl'
classifys = pickle.load(open(filename, 'rb'))
filename = 'datafeatures.pkl'
datafeatures = pickle.load(open(filename, 'rb'))

In [ ]:
cd 'C:\Python27\Notebooks\wfdb\apneaecg\Labelled1'

In [ ]:
dflen = len(datafeatures)
split = (9*dflen/10)     

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

In [ ]:
svm = SVC(C=1.0, gamma=0.1, kernel='rbf')
svm.fit(datafeatures[:split], classifys[:split])        # We're training on just the dataset up to the split

# Make an array of predictions on the validation set
predictions = svm.predict(datafeatures[split+1:dflen-1])        # getting predictions based on the held back validation set

# Output the percentage accuracy, total predicted correctly and the confusion matrix for each model
print(svm.score(datafeatures[split+1:dflen-1], classifys[split+1:dflen-1]))
print(confusion_matrix(predictions, classifys[split+1:dflen-1]))

In [ ]:
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn import linear_model, metrics
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

## Random Forest Classifier

In [ ]:
classifier = RandomForestClassifier()

classifier.fit(datafeatures[:split], classifys[:split])

# Make an array of predictions on the validation set
predictions = classifier.predict(datafeatures[split+1:dflen-1])

# Output the percentage accuracy, total predicted correctly and the confusion matrix for each model
print(classifier.score(datafeatures[split+1:dflen-1], classifys[split+1:dflen-1]))
print(confusion_matrix(predictions, classifys[split+1:dflen-1]))

In [ ]:
import pickle
filename = 'RF_Class.pkl'
pickle.dump(classifier, open(filename, 'wb'), protocol=2)

## SVM SVC(C=10, gamma=0.001, kernel='rbf')

In [ ]:
classifier = SVC(C=10, gamma=0.001, kernel='rbf')

classifier.fit(datafeatures[:split], classifys[:split])

# Make an array of predictions on the validation set
predictions = classifier.predict(datafeatures[split+1:dflen-1])

# Output the percentage accuracy, total predicted correctly and the confusion matrix for each model
print(classifier.score(datafeatures[split+1:dflen-1], classifys[split+1:dflen-1]))
print(confusion_matrix(predictions, classifys[split+1:dflen-1]))

## K-Nearest Neighbour

In [ ]:
classifier = KNeighborsClassifier()

classifier.fit(datafeatures[:split], classifys[:split])

# Make an array of predictions on the validation set
predictions = classifier.predict(datafeatures[split+1:dflen-1])

# Output the percentage accuracy, total predicted correctly and the confusion matrix for each model
print(classifier.score(datafeatures[split+1:dflen-1], classifys[split+1:dflen-1]))
print(confusion_matrix(predictions, classifys[split+1:dflen-1]))

In [ ]:
import pickle
pickle.dump(classifier, open('KNear.pkl', 'wb'), protocol=0)  



## Decision Tree

In [ ]:
classifier = DecisionTreeClassifier()

classifier.fit(datafeatures[:split], classifys[:split])

# Make an array of predictions on the validation set
predictions = classifier.predict(datafeatures[split+1:dflen-1])

# Output the percentage accuracy, total predicted correctly and the confusion matrix for each model
print(classifier.score(datafeatures[split+1:dflen-1], classifys[split+1:dflen-1]))
print(confusion_matrix(predictions, classifys[split+1:dflen-1]))